In [ ]:
from scwidgets.code import CodeInput, ParametersPanel
from scwidgets.cue import CueFigure
from scwidgets.exercise import CodeExercise, TextExercise

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from ipywidgets import Layout

In [ ]:
from scwidgets.code import CodeInput, ParametersPanel
from scwidgets.cue import CueFigure
from scwidgets.exercise import CodeExercise, TextExercise
from ipywidgets import IntSlider, FloatSlider

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
TextExercise(
    title="Derive solution for weights",
    description="""
    <p style="width: 500px;">We can define ridge regression by extending the ordinary least
    square solution by a penalization $\\lambda\\|\\mathbf{w}\\|_2^2$. Please derive the solution
    for the weights from the optimization problem:</p>
    $$\\hat{\\mathbf{w}} = \\min_\\mathbf{w} \\|\\mathbf{y}-\\mathbf{X}\\mathbf{w}\\|^2 + \\lambda\\|\\mathbf{w}\\|^2$$""",
    layout=Layout(width="430px", height="50px")
)

In [ ]:
from scwidgets.code import CodeInput, ParametersPanel
from scwidgets.cue import CueFigure
from scwidgets.exercise import CodeExercise
from ipywidgets import IntSlider, FloatSlider

import matplotlib.pyplot as plt
import numpy as np

# This is what the students sees and can adapt
def compute_weigths(X, y, regularizer):
    """Implements ridge regression"""
    from numpy.linalg import inv
    return inv(X.T@X + regularizer) @ X.T @ y

code_input = CodeInput(compute_weigths, builtins={'np': np})

#ex_figure, _ = plt.subplots(1, 1, figsize=(7.5, 3.8))
ex_figure = plt.figure(tight_layout=True, figsize=(4.5, 3.5))
# We wrap figures so we can enforce a behaviour that is conform with the
# CodeExercise widget.
cue_ex_figure = CueFigure(ex_figure)

# We wrap figures so we can enforce a behaviour that is conform with the
# CodeExercise widget.
cue_ex_figure = CueFigure(ex_figure)
learning_figure = CueFigure(plt.figure(tight_layout=True, figsize=(4.5, 3.5)))


rng = np.random.default_rng(seed=1)
#x = rng.standard_normal((50, 1))
x = rng.random((100, 1)) *4 -2
weights = rng.standard_normal(5)
y_true = np.hstack([x**degree for degree in range(1,6)]) @ weights 
x_grid = np.linspace(-2., 2., 100)
xstack = np.stack([x_grid**degree for degree in range(1,6)]).T
y_grid = xstack @ weights
# This is hidden to the student, so we can do the pre- and postprocessing

parameter_panel = ParametersPanel(regularizer_log10=IntSlider(-1, -9, 9, step=3, description="$\\normalsize\\log_{10}\\lambda$"),
                                 noise_sigma=FloatSlider(value=5.0, min=0.0, max=20, step=1, description="$\\normalsize\\sigma$"),
                                 degree_max=IntSlider(value=2, min=0, max=10, step=1, description="$\\normalsize d_\\text{max}$"),
                                 nb_samples=IntSlider(value=20, min=10, max=100, step=1, description="$\\normalsize n_\\text{samples}$"))
def rmse(y_pred, y_true):
    return np.sqrt(np.mean((y_pred-y_true)**2))

# this is hidden to the student 
def ex_updater():
    nb_samples = parameter_panel.panel_parameters["nb_samples"]
    X = np.hstack([x[:nb_samples]**degree for degree in range(parameter_panel.panel_parameters["degree_max"]+1)])
    noise_sigma = parameter_panel.panel_parameters["noise_sigma"]
    regularizer = 10**parameter_panel.panel_parameters["regularizer_log10"]
    rng = np.random.default_rng(seed=5)
    noise = rng.normal(loc=0, scale=noise_sigma, size=y_true.shape)
    y_true_ = y_true[:nb_samples]
    noise = noise[:nb_samples]
    y_noisy = y_true_ + noise
    weight_pred = code_input.get_function_object()(X, y_noisy, regularizer)
    y_pred = X@weight_pred
    xstack = np.stack([x_grid**degree for degree in range(parameter_panel.panel_parameters["degree_max"]+1)]).T
    y_grid_pred = xstack@weight_pred

    
    cue_ex_figure.clear_figure()
    ax = cue_ex_figure.figure.gca()
    #ax.plot([-10, 10], [-10, 10], color='black', alpha=0.5)
    print(f"train error: {rmse(y_pred, y_noisy):.2f}, test error: {rmse(y_pred, y_true_):.2f}")
    ax.plot(x_grid, y_grid, linestyle="-", label="$f(x)$")
    ax.scatter(x[:nb_samples], y_noisy, label="$f(x_i)+\\epsilon_i$")
    
    ax.plot(x_grid, y_grid_pred, label="$\\hat{f}(x)$")
    #ax.plot([-500], [-500], label="$g(x)$", marker='o', color='C1')
    ax.scatter(x[:nb_samples], y_pred, label="$\\hat{f}(x_i)$")
    #ax.set_title("parity plot")
    #ax.scatter(y, y_true_)
    ax.set_xlabel("$x$")
    ax.set_ylabel("$f(x)$")
    ax.set_ylim(np.min(y_true), np.max(y_true))
    ax.set_xlim(np.min(x), np.max(x))
    ax.legend()
    
code_ex = CodeExercise(
    code=code_input,
    parameters=parameter_panel,
    outputs=cue_ex_figure,
    update=ex_updater,
    update_mode="continuous",
    title="Ridge regression",
    description="""
        <p style="width: 500px;">
        Implements the computation of the weights in ridge regression for the regularization parameter $\\lambda$.
        The ground truth function is $f(x) = 0.33x-0.65x^2+0.86x^3-0.12x^4+0.66x^5$.
        The ground truth labels are added with a noise $f(x_i) + \\epsilon_i = y^\\textrm{true}_i + \\epsilon_i$ that are sampled from a Gaussian
        distribution $\\epsilon_i\\sim\\mathcal{N}(0, \\sigma^2)$.
        """
)


code_ex.run_update()
code_ex

In [ ]:
TextExercise(
    title="Questions",
    description="""
    <p style="width: 500px;">For which parameters a nearly exact solution can be reproduced?
    What other relationships can you observe?""",
    layout=Layout(width="430px", height="50px")
)